
# Praktikum Datavis – Chapter 3 (From Static to Interactive Visualization)

Notebook ini dirancang **satu file saja** agar alur tidak membingungkan. Kita mulai dari **plot statis** lalu naik ke **interaktif** sesuai _Chapter 3_ pada bahan ajar.

## Tujuan Praktikum
1. Memahami perbedaan **visualisasi statis vs interaktif** secara praktis.
2. Membuat plot **Bokeh** dengan **slider** (filter) dan **hover** (tooltip).
3. Membuat plot **Plotly Express** yang interaktif dan **animasi** sederhana.



## 0) Setup Lingkungan (dibaca dulu)
- Disarankan pakai **venv**:
  - Windows PowerShell: `python -m venv venv && .\venv\Scripts\Activate.ps1`
  - macOS/Linux: `python -m venv venv && source venv/bin/activate`
- Install paket minimal:
  ```bash
  pip install notebook jupyterlab pandas matplotlib seaborn bokeh plotly
  ```
- Jalankan: `jupyter lab` atau `jupyter notebook` lalu buka file ini.



## 1) Cek Library & Versi
Jalankan sel ini untuk memastikan dependensi tersedia.


In [1]:

import sys, platform
import pandas as pd, seaborn as sns, matplotlib, matplotlib.pyplot as plt
import bokeh
import plotly

print("Python  :", sys.version.split()[0])
print("OS      :", platform.platform())
print("pandas  :", pd.__version__)
print("seaborn :", sns.__version__)
print("matplotlib:", matplotlib.__version__)
print("bokeh   :", bokeh.__version__)
print("plotly  :", plotly.__version__)

# sanity check dataset bawaan seaborn
df_demo = sns.load_dataset('diamonds')
df_demo.head()


Python  : 3.12.3
OS      : Linux-6.6.87.2-microsoft-standard-WSL2-x86_64-with-glibc2.39
pandas  : 2.3.2
seaborn : 0.13.2
matplotlib: 3.10.6
bokeh   : 3.8.0
plotly  : 6.3.0


,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75



## 2) Static vs Interactive (konsep singkat)
- **Statis**: gambar tetap (contoh: PNG/PDF dari matplotlib/seaborn). Cocok untuk laporan.
- **Interaktif**: merespons input user (zoom, hover, filter, animasi) → eksplorasi data & dashboard.

Di bawah ini, kita langsung praktik: mulai dari plot statis → lalu kita tambahkan **slider** dan **hover** dengan **Bokeh**.



## 3) Bokeh – Baseline Static Plot (Exercise 22–23)
Kita gunakan dataset **Iris**. Pertama buat scatter plot sebagai baseline (belum interaktif).


In [2]:
import seaborn as sns
from bokeh.plotting import figure, output_notebook, show

output_notebook()

iris = sns.load_dataset("iris")

p = figure(
    title="Iris Scatter (Static Baseline)",
    x_axis_label="Petal Length",
    y_axis_label="Petal Width",
    width=700,
    height=420,
)
p.circle(iris["petal_length"], iris["petal_width"], size=8, color="navy", alpha=0.5)

show(p)

Loading BokehJS ...


## 4) Bokeh – Tambah **Slider** & **Hover** (Exercise 24–25)
> Catatan penting: Callback **Python** untuk `Slider.on_change` butuh **Bokeh Server**.  
> Agar tetap **satu notebook** (tanpa server), kita gunakan **CustomJS** (JavaScript) untuk memfilter data di sisi browser.

**Target interaksi:**
- Slider membatasi **maksimum `petal_length`** yang ditampilkan.
- Hover menampilkan `species`, `petal_length`, `petal_width`.


In [3]:
import seaborn as sns
from bokeh.models import ColumnDataSource, Slider, HoverTool, CustomJS
from bokeh.layouts import column
from bokeh.plotting import figure, show

iris = sns.load_dataset("iris")

source_full = ColumnDataSource(data=dict(
    petal_length=iris["petal_length"].tolist(),
    petal_width=iris["petal_width"].tolist(),
    species=iris["species"].astype(str).tolist(),
))

source_view = ColumnDataSource(data=dict(source_full.data))

p2 = figure(
    title="Iris with Slider (CustomJS) & Hover",
    x_axis_label="Petal Length",
    y_axis_label="Petal Width",
    width=700,
    height=420,
    tools="pan,wheel_zoom,box_zoom,reset,save",
)
p2.circle("petal_length", "petal_width", source=source_view, size=8, alpha=0.6, color="teal")

hover = HoverTool(tooltips=[
    ("Species", "@species"),
    ("Petal Len", "@petal_length"),
    ("Petal Wid", "@petal_width"),
])
p2.add_tools(hover)

slider = Slider(
    start=min(source_full.data["petal_length"]),
    end=max(source_full.data["petal_length"]),
    value=max(source_full.data["petal_length"]),
    step=0.1,
    title="Max Petal Length",
)

callback = CustomJS(args=dict(s_full=source_full, s_view=source_view, sl=slider), code="""
    const full = s_full.data;
    const view = s_view.data;
    const limit = sl.value;

    const pl = [];
    const pw = [];
    const sp = [];

    for (let i = 0; i < full.petal_length.length; i++) {
        if (full.petal_length[i] <= limit) {
            pl.push(full.petal_length[i]);
            pw.push(full.petal_width[i]);
            sp.push(full.species[i]);
        }
    }
    view.petal_length = pl;
    view.petal_width = pw;
    view.species = sp;
    s_view.change.emit();
""")
slider.js_on_change("value", callback)

show(column(p2, slider))


## 5) Plotly Express – Scatter Interaktif (Exercise 26)
**Plotly Express** sangat ringkas: 1–2 baris sudah dapat plot interaktif (zoom, pan, hover, unduh gambar).


In [4]:

import plotly.express as px
df_iris = px.data.iris()

fig = px.scatter(df_iris, x="sepal_length", y="sepal_width",
                 color="species", size="petal_length",
                 hover_data=['petal_width'],
                 title="Iris Scatter – Plotly Express")
fig.show()



## 6) Plotly Express – Animasi (Activity 3)
Contoh animasi menggunakan dataset bawaan **gapminder**. Perhatikan kontrol animasi di kanan-bawah grafik.


In [5]:

import plotly.express as px
gap = px.data.gapminder()

fig = px.scatter(gap, x="gdpPercap", y="lifeExp",
                 size="pop", color="continent",
                 hover_name="country", log_x=True,
                 animation_frame="year", animation_group="country",
                 title="Gapminder: GDP vs Life Expectancy (Animated)")
fig.show()



## 7) Refleksi Cepat
- Kapan **statis** lebih tepat? Kapan **interaktif** lebih bermakna?
- Kapan memilih **Bokeh** (fleksibel, kustom) vs **Plotly** (ringkas, cepat)?
- Interaksi minimum apa yang paling membantu audiens (hover? filter? animasi? zoom?)

## 8) Tugas Mini (opsional)
1. Ganti dataset (misal `tips` dari seaborn): buat **1 plot Bokeh** dengan slider & hover.
2. Buat **1 plot Plotly** lain (bar/line/density contour) dengan warna per kategori.
3. Tuliskan **2–3 poin insight** dari visual yang kamu buat.
